# ProT-Diff 模型 - 面向过程编程

理解ProT-Diff模型的训练过程，用函数先代替复杂的类结构。

## 模型概述
ProT-Diff是一个用于生成抗菌肽(AMP)的扩散模型：
1. 使用ProtT5编码器将肽序列转换为嵌入向量
2. 在嵌入空间训练扩散过程
3. 从训练好的扩散模型采样新的嵌入
4. 使用ProtT5解码器将嵌入转换回氨基酸序列


## 第1步：环境设置和依赖导入


In [2]:
# 环境设置说明：
# 如果遇到numpy兼容性问题，请运行以下命令修复：
# conda install numpy=1.24.3 scipy=1.10.1 scikit-learn=1.3.0 -c conda-forge --yes
# 安装必要的包（如果还没安装）：
# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
# conda install transformers accelerate einops pandas scikit-learn tqdm matplotlib -c conda-forge

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import T5Tokenizer, T5EncoderModel, T5ForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput
from einops import rearrange

# 检查版本兼容性
print(f"NumPy版本: {np.__version__}")
print(f"PyTorch版本: {torch.__version__}")
try:
    import transformers
    print(f"Transformers版本: {transformers.__version__}")
except:
    print("Transformers版本: 导入失败")

# 设置设备
device = "cuda" if torch.cuda.is_available() else "mps"
print(f"使用设备: {device}")

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)


NumPy版本: 1.26.4
PyTorch版本: 2.8.0+cu128
Transformers版本: 4.42.4
使用设备: cuda


## 第2步：准备训练数据

我们支持多种数据源：
- **演示数据**：8条示例序列，用于快速学习
- **AMP数据集**：真实的抗菌肽序列数据，用于finetuning
- **Non-AMP数据集**：非抗菌肽序列

### 数据要求：
- 序列长度：5-100个氨基酸
- 只包含20种标准氨基酸：ACDEFGHIKLMNPQRSTVWY



In [6]:
# 示例抗菌肽序列（长度5-48，只包含20种氨基酸）
example_sequences = [
    "GIGKFLKKAKKFGKAFVKILKK",  # 22个氨基酸
    "KKLFKKILKYL",             # 11个氨基酸
    "GLFDIVKKVVGAL",           # 13个氨基酸
    "RWKIFKKIERVGQHTRDAT",     # 19个氨基酸
    "KWKLFKKIPKFLHLAKKF",      # 18个氨基酸
    "FLPIIAKLLSGLL",           # 13个氨基酸
    "KLAKLAKKLAKLAK",          # 14个氨基酸
    "GIGAVLKVLTTGLPALIS"       # 18个氨基酸
]

def load_sequences(use_example=True, dataset_path=None):
    """
    选择示例数据或加载自定义数据集。
    dataset_path 需要是一个包含肽序列的文件（csv）。
    """
    if use_example:
        return example_sequences
    else:
        if dataset_path is None:
            raise ValueError("请提供 dataset_path 参数来加载自定义数据集")
        if dataset_path.endswith(".csv"):
            try:
                df = pd.read_csv(dataset_path)
                # 如果有'sequence'列，使用它；否则使用第一列
                if 'sequence' in df.columns:
                    sequences = df['sequence'].tolist()
                else:
                    sequences = df.iloc[:, 0].tolist()
                # 过滤掉空值和非字符串值
                sequences = [str(seq) for seq in sequences if pd.notna(seq) and str(seq).strip() != '']
                return sequences
            except Exception as e:
                print(f"读取CSV文件时出错: {e}")
                print("尝试使用更简单的方法读取...")
                # 备用方法：直接按行读取
                with open(dataset_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    # 跳过标题行，提取第一列
                    sequences = []
                    for i, line in enumerate(lines):
                        if i == 0:  # 跳过标题
                            continue
                        parts = line.strip().split(',')
                        if len(parts) > 0 and parts[0]:
                            sequences.append(parts[0])
                    return sequences
        else:
            raise ValueError("目前仅支持 csv 格式文件")

# ===== 使用示例数据 =====
#sequences = load_sequences(use_example=True)

# ===== 或者使用你自己的数据集 =====
sequences = load_sequences(use_example=False, dataset_path="data/Non-AMP/final_non_amps.csv")

print("展示 5 条序列：")
sample_seqs = sequences[:5]
for i, seq in enumerate(sample_seqs, 1):
    print(f"{i}. {seq} (长度: {len(seq)})")

# 统计信息
lengths = [len(seq) for seq in sequences]
total_count = len(sequences)
max_len = max(lengths)
min_len = min(lengths)
median_len = np.median(lengths)
mean_len = np.mean(lengths)
print(f"\n总共 {total_count} 条序列")
print(f"最大长度: {max_len}")
print(f"最小长度: {min_len}")
print(f"中位数长度: {median_len:.2f}")
print(f"平均长度: {mean_len:.2f}")


读取CSV文件时出错: Cannot convert numpy.ndarray to numpy.ndarray
尝试使用更简单的方法读取...
展示 5 条序列：
1. MLSCWKGYI (长度: 9)
2. GSPPVFDRVVVNPQLYENRLNQTRLGT (长度: 27)
3. MIIWLPPLVAAVTTYLLCEYLYYYGRDEH (长度: 29)
4. MEIEDRIDLSERGHDTLEQKR (长度: 21)
5. PVWIMAHMVNAVAQIDEFVNL (长度: 21)

总共 108776 条序列
最大长度: 100
最小长度: 4
中位数长度: 35.00
平均长度: 31.79


## 第3步：加载ProtT5模型

ProtT5是一个预训练的蛋白质语言模型，我们需要加载编码器和解码器：
- **编码器**：将氨基酸序列转换为嵌入向量
- **解码器**：将嵌入向量转换回氨基酸序列
cd /root/clash
nohup ./clash -d . > clash.log 2>&1 &
export http_proxy=http://127.0.0.1:7890
export https_proxy=http://127.0.0.1:7890
curl -I https://www.google.com

In [7]:
import os, re, math
from tqdm.auto import tqdm

from typing import List
from torch.utils.data import Dataset, DataLoader
# ====== 配置 ======
MODEL_NAME = "Rostlab/prot_t5_xl_half_uniref50-enc"  # 半精度 encoder-only
TOKENIZER_NAME = "Rostlab/prot_t5_xl_uniref50"       # Tokenizer 用全量版
MAX_LEN = 48
BATCH_SIZE = 16
DEVICE = device  # 使用之前定义的device

# ====== 实用函数 ======
AA_20 = set("ACDEFGHIKLMNPQRSTVWY")
def clean_seq(seq: str) -> str:
    s = re.sub(r"[^A-Z]", "", seq.upper())
    s = "".join([c for c in s if c in AA_20])
    return s

def space_separate(seq: str) -> str:
    return " ".join(list(seq))

In [6]:
# ==== 0) 安装依赖（已装过可跳过） ====
%env HF_HUB_ENABLE_HF_TRANSFER=1
import os
print("HF_HUB_ENABLE_HF_TRANSFER =", os.environ.get("HF_HUB_ENABLE_HF_TRANSFER"))

from huggingface_hub import constants
print("HF transfer enabled? ", constants.HF_HUB_ENABLE_HF_TRANSFER)  


env: HF_HUB_ENABLE_HF_TRANSFER=1
HF_HUB_ENABLE_HF_TRANSFER = 1
HF transfer enabled?  False


In [7]:
import os, pathlib, time
from huggingface_hub import snapshot_download

# 0) 缓存目录
os.environ["HF_HOME"] = "/root/autodl-tmp/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/huggingface/transformers"
pathlib.Path(os.environ["HF_HOME"]).mkdir(parents=True, exist_ok=True)
pathlib.Path(os.environ["TRANSFORMERS_CACHE"]).mkdir(parents=True, exist_ok=True)

# 1) 代理
proxy = "http://127.0.0.1:7890"
for var in ["HTTP_PROXY","HTTPS_PROXY","http_proxy","https_proxy"]:
    os.environ[var] = proxy

# 2) 加速 + 禁用 Xet
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["HF_HUB_DISABLE_XET"] = "1"  # ← 强制禁用 Xet
os.environ["HF_HUB_ETAG_TIMEOUT"] = "60"

# 3) 下载
repo_id = "Rostlab/prot_t5_xl_uniref50"
dest_dir = "/root/autodl-tmp/prot_t5_xl_uniref50"
token = None  # 如果需要登录，改成你的 token

retries, delay_s = 5, 15
last_err = None

for i in range(1, retries + 1):
    try:
        local_dir = snapshot_download(
            repo_id=repo_id,
            local_dir=dest_dir,
            resume_download=True,
            max_workers=1,
            etag_timeout=60,
            token=token
        )
        print(f"Model stored at: {local_dir}")
        last_err = None
        break
    except Exception as e:
        last_err = e
        print(f"[retry {i}/{retries}] {e}")
        time.sleep(delay_s)

if last_err:
    raise last_err


/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Model stored at: /root/autodl-tmp/prot_t5_xl_uniref50


In [8]:
# ==== 4) 加载 Tokenizer 与 Encoder（直接用本地缓存路径） ====
import re, math
from transformers import T5Tokenizer, T5EncoderModel
import torch

# 设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 本地模型路径（替换成你的实际下载目录）
LOCAL_MODEL_DIR = "/root/autodl-tmp/prot_t5_xl_uniref50"

print("正在加载 ProtT5 编码器/Tokenizer ...")

# 直接从本地路径加载
tokenizer = T5Tokenizer.from_pretrained(LOCAL_MODEL_DIR, do_lower_case=False)
encoder_model = T5EncoderModel.from_pretrained(LOCAL_MODEL_DIR)

# 精度与设备
if device == "cuda":
    # prot_t5_xl 较大，半精度可显著省显存
    encoder_model = encoder_model.half().to(device)
else:
    encoder_model = encoder_model.float().to(device)

encoder_model.eval()

print("✓ ProtT5 编码器加载成功")
print(f"✓ 参数量: {sum(p.numel() for p in encoder_model.parameters()):,}")
print(f"✓ 嵌入维度 d_model: {encoder_model.config.d_model}")
print(f"✓ 使用设备: {device}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


正在加载 ProtT5 编码器/Tokenizer ...
✓ ProtT5 编码器加载成功
✓ 参数量: 1,208,141,824
✓ 嵌入维度 d_model: 1024
✓ 使用设备: cuda


## 第4步：序列编码 - 将氨基酸序列转换为嵌入向量

这一步我们要实现：
1. 单个序列编码函数：将氨基酸序列编码为(L,1024)的嵌入
2. 零填充到固定长度：填充到(48,1024)的固定形状
3. 批量编码：处理所有训练序列


In [9]:
# ====== 单序列编码（用于快速测试） ======
@torch.inference_mode()
def encode_sequence(sequence: str,
                    tokenizer: T5Tokenizer,
                    encoder_model: T5EncoderModel,
                    device: str,
                    max_length: int = MAX_LEN) -> tuple[torch.Tensor, int]:
    """
    编码单个序列，返回嵌入和真实长度
    
    Returns:
        tuple: (embeddings, real_length)
            - embeddings: (real_length, d_model) 真实长度的嵌入，不做填充
            - real_length: int 序列的真实长度
    """
    seq = clean_seq(sequence)
    real_length = len(seq)
    assert 1 <= real_length <= max_length, f"序列长度必须 1~{max_length}，当前 {real_length}"
    
    spaced = space_separate(seq)
    inputs = tokenizer(
        spaced,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length + 2,  # 预留 special tokens
        add_special_tokens=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = encoder_model(**inputs).last_hidden_state.squeeze(0)  # (seq_len_w_special, d_model)
    
    # 去掉首尾 special tokens，保留真实长度
    outputs = outputs[1:real_length+1]  # (real_length, d_model)
    
    return outputs, real_length  # (real_length, d_model), int

# ====== 批量编码（推荐） ======
class SeqDataset(Dataset):
    def __init__(self, sequences: List[str]):
        self.seqs = [clean_seq(s) for s in sequences]
        # 过滤过短/过长
        self.seqs = [s for s in self.seqs if 1 <= len(s) <= MAX_LEN]
    def __len__(self):
        return len(self.seqs)
    def __getitem__(self, idx):
        return self.seqs[idx]

def collate_fn(batch: List[str]):
    # 把 batch 的序列拼成 spaced 文本，交给 tokenizer 做 padding
    spaced = [space_separate(s) for s in batch]
    enc = tokenizer(
        spaced,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LEN + 2,
        add_special_tokens=True
    )
    return enc, batch  # 返回原序列以便调试

@torch.inference_mode()
def encode_batch(sequences: List[str],
                 encoder_model: T5EncoderModel,
                 batch_size: int = BATCH_SIZE,
                 device: str = DEVICE,
                 max_length: int = MAX_LEN) -> tuple[List[torch.Tensor], List[int]]:
    """
    批量编码序列，返回嵌入列表和长度列表
    
    Returns:
        tuple: (embeddings_list, lengths_list)
            - embeddings_list: List[torch.Tensor] 每个元素形状为 (real_length, d_model)
            - lengths_list: List[int] 每个序列的真实长度
    """
    ds = SeqDataset(sequences)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    embs_list = []
    lengths_list = []
    
    for inputs, raw_batch in tqdm(dl, desc="批量编码", leave=False):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        out = encoder_model(**inputs).last_hidden_state  # (B, Lw, d_model)
        
        # 为每个序列单独处理，保留真实长度
        for i, raw_seq in enumerate(raw_batch):
            real_length = len(raw_seq)
            lengths_list.append(real_length)
            
            # 去掉 special tokens，保留真实长度的嵌入
            seq_emb = out[i, 1:real_length+1, :]  # (real_length, d_model)
            
            # 移到 CPU 节省显存
            embs_list.append(seq_emb.float().cpu())
    
    return embs_list, lengths_list

# ====== 辅助函数：如需固定长度可使用此函数 ======
def pad_embeddings_to_fixed_length(embs_list: List[torch.Tensor], 
                                   lengths_list: List[int],
                                   target_length: int = MAX_LEN) -> torch.Tensor:
    """
    将变长嵌入列表填充到固定长度（向后兼容）
    
    Args:
        embs_list: 变长嵌入列表
        lengths_list: 对应的长度列表
        target_length: 目标长度
        
    Returns:
        torch.Tensor: (N, target_length, d_model) 固定长度的嵌入张量
    """
    if not embs_list:
        return torch.empty(0, target_length, embs_list[0].shape[-1])
    
    d_model = embs_list[0].shape[-1]
    N = len(embs_list)
    
    # 创建固定长度的张量
    padded_embs = torch.zeros(N, target_length, d_model, dtype=embs_list[0].dtype)
    
    for i, (emb, length) in enumerate(zip(embs_list, lengths_list)):
        actual_length = min(length, target_length)
        padded_embs[i, :actual_length] = emb[:actual_length]
    
    return padded_embs

# ====== 快速自测 ======
if 'example_sequences' not in globals():
    example_sequences = ["GIGKFLKKAKKFGKAFVKILKK", "LLKKLLKKLLKKLL"]
if 'sequences' not in globals():
    sequences = example_sequences * 10

print("\n[单条编码自测]")
emb1, length1 = encode_sequence(example_sequences[0], tokenizer, encoder_model, DEVICE, MAX_LEN)
print(f"序列: {example_sequences[0]}")
print(f"真实长度: {length1}")
print(f"嵌入形状: {tuple(emb1.shape)}")
print(f"数据类型: {emb1.dtype}")
print(f"数值范围: [{float(emb1.min()):.6f}, {float(emb1.max()):.6f}]")

print("\n[批量编码自测]")
test_seqs = example_sequences[:3]  # 测试前3个序列
embs_list, lengths_list = encode_batch(test_seqs, encoder_model, 2, DEVICE, MAX_LEN)
print(f"编码了 {len(embs_list)} 个序列")
for i, (emb, length) in enumerate(zip(embs_list, lengths_list)):
    print(f"序列 {i+1}: 长度={length}, 嵌入形状={tuple(emb.shape)}")
    print(f"  原始序列: {test_seqs[i][:20]}{'...' if len(test_seqs[i]) > 20 else ''}")

print("\n[向后兼容性测试 - 固定长度填充]")
padded_embs = pad_embeddings_to_fixed_length(embs_list, lengths_list, MAX_LEN)
print(f"固定长度嵌入形状: {tuple(padded_embs.shape)}")
print("✓ 所有测试通过！现在编码函数会保留真实长度信息")



[单条编码自测]
序列: GIGKFLKKAKKFGKAFVKILKK
真实长度: 22
嵌入形状: (22, 1024)
数据类型: torch.float16
数值范围: [-0.893066, 0.850586]

[批量编码自测]


批量编码:   0%|          | 0/2 [00:00<?, ?it/s]

编码了 3 个序列
序列 1: 长度=22, 嵌入形状=(22, 1024)
  原始序列: GIGKFLKKAKKFGKAFVKIL...
序列 2: 长度=11, 嵌入形状=(11, 1024)
  原始序列: KKLFKKILKYL
序列 3: 长度=13, 嵌入形状=(13, 1024)
  原始序列: GLFDIVKKVVGAL

[向后兼容性测试 - 固定长度填充]
固定长度嵌入形状: (3, 48, 1024)
✓ 所有测试通过！现在编码函数会保留真实长度信息


## ✅ 编码函数修改完成

### 🔄 主要变化：

1. **`encode_sequence()`** 现在返回 `(embeddings, real_length)`
   - `embeddings`: 形状为 `(real_length, d_model)` 的真实长度嵌入
   - `real_length`: 序列的实际长度

2. **`encode_batch()`** 现在返回 `(embeddings_list, lengths_list)`
   - `embeddings_list`: 每个元素为 `(real_length, d_model)` 的嵌入列表
   - `lengths_list`: 对应的真实长度列表

3. **新增辅助函数 `pad_embeddings_to_fixed_length()`**
   - 用于向后兼容，将变长嵌入填充到固定长度
   - 返回 `(N, target_length, d_model)` 的固定形状张量

### 💡 使用示例：

```python
# 单序列编码 - 保留真实长度
emb, length = encode_sequence("GIGKFLKKAKKF", tokenizer, encoder_model, device)
print(f"嵌入形状: {emb.shape}")  # (12, 1024) 而不是 (48, 1024)

# 批量编码 - 保留所有真实长度
embs_list, lengths = encode_batch(sequences, encoder_model)
# 每个 embs_list[i] 的形状都是 (lengths[i], 1024)

# 如需固定长度（向后兼容）
fixed_embs = pad_embeddings_to_fixed_length(embs_list, lengths, 48)
print(f"固定长度形状: {fixed_embs.shape}")  # (N, 48, 1024)
```

### ✨ 优势：

- **内存效率**：不浪费空间在填充的零向量上
- **真实信息**：保留每个序列的实际长度
- **灵活性**：可以根据需要选择是否使用固定长度
- **向后兼容**：提供辅助函数支持原有的固定长度需求


In [13]:
# 批量编码所有训练序列
print("正在编码所有训练序列...")

# 使用高效批量编码（简化格式：直接保存嵌入列表）
if os.path.exists("embedding_non_amp.pt"):
    train_embeddings_list = torch.load("embedding_non_amp.pt", map_location="cpu")
    train_lengths = [emb.size(0) for emb in train_embeddings_list]  # 从嵌入推断长度
    print(f"✓ 从缓存加载了 {len(train_embeddings_list)} 个序列的嵌入")
else:
    train_embeddings_list, train_lengths = encode_batch(sequences, encoder_model, BATCH_SIZE, DEVICE, MAX_LEN)
    print(f"✓ 所有序列编码完成!")
    print(f"✓ 编码序列数量: {len(train_embeddings_list)}")
    print(f"✓ 长度统计: 最小={min(train_lengths)}, 最大={max(train_lengths)}, 平均={sum(train_lengths)/len(train_lengths):.1f}")
    
    # 检查数据类型和范围
    if train_embeddings_list:
        sample_emb = train_embeddings_list[0]
        print(f"✓ 数据类型: {sample_emb.dtype}")
        all_values = torch.cat(train_embeddings_list, dim=0)
        print(f"✓ 数值范围: [{all_values.min():.3f}, {all_values.max():.3f}]")

    # 保存嵌入数据（简化格式：只保存嵌入列表）
    print("\n正在保存嵌入数据...")
    torch.save(train_embeddings_list, "embedding_non_amp.pt")
    print(f"✓ 嵌入数据已保存到: embedding_non_amp.pt")
    
    # 计算总文件大小
    total_elements = sum(emb.numel() for emb in train_embeddings_list)
    element_size = train_embeddings_list[0].element_size() if train_embeddings_list else 4
    file_size_mb = total_elements * element_size / 1024 / 1024
    print(f"✓ 文件大小: {file_size_mb:.1f} MB")

# # 如果需要固定长度格式（向后兼容）
# print("\n[可选] 转换为固定长度格式:")
# train_embeddings_fixed = pad_embeddings_to_fixed_length(train_embeddings_list, train_lengths, MAX_LEN)
# print(f"✓ 固定长度嵌入形状: {train_embeddings_fixed.shape}")
# print("✓ 现在您可以选择使用变长格式（train_embeddings_list, train_lengths）或固定长度格式（train_embeddings_fixed）")


正在编码所有训练序列...


批量编码:   0%|          | 0/6231 [00:00<?, ?it/s]

✓ 所有序列编码完成!
✓ 编码序列数量: 99687
✓ 长度统计: 最小=4, 最大=48, 平均=30.0
✓ 数据类型: torch.float32
✓ 数值范围: [-1.900, 1.873]

正在保存嵌入数据...
✓ 嵌入数据已保存到: embedding_non_amp.pt
✓ 文件大小: 11678.0 MB


In [14]:
sequence = load_sequences(use_example=False, dataset_path="data/AMP/final_AMP.csv")
print(len(sequence))

读取CSV文件时出错: Cannot convert numpy.ndarray to numpy.ndarray
尝试使用更简单的方法读取...
7956


In [15]:
# 批量编码所有训练序列
print("正在编码所有训练序列...")

# 使用高效批量编码（简化格式：直接保存嵌入列表）
if os.path.exists("embedding_amp.pt"):
    train_embeddings_list = torch.load("embedding_amp.pt", map_location="cpu")
    train_lengths = [emb.size(0) for emb in train_embeddings_list]  # 从嵌入推断长度
    print(f"✓ 从缓存加载了 {len(train_embeddings_list)} 个序列的嵌入")
else:
    train_embeddings_list, train_lengths = encode_batch(sequence, encoder_model, BATCH_SIZE, DEVICE, MAX_LEN)
    print(f"✓ 所有序列编码完成!")
    print(f"✓ 编码序列数量: {len(train_embeddings_list)}")
    print(f"✓ 长度统计: 最小={min(train_lengths)}, 最大={max(train_lengths)}, 平均={sum(train_lengths)/len(train_lengths):.1f}")
    
    # 检查数据类型和范围
    if train_embeddings_list:
        sample_emb = train_embeddings_list[0]
        print(f"✓ 数据类型: {sample_emb.dtype}")
        all_values = torch.cat(train_embeddings_list, dim=0)
        print(f"✓ 数值范围: [{all_values.min():.3f}, {all_values.max():.3f}]")

    # 保存嵌入数据（简化格式：只保存嵌入列表）
    print("\n正在保存嵌入数据...")
    torch.save(train_embeddings_list, "embedding_amp.pt")
    print(f"✓ 嵌入数据已保存到: embedding_amp.pt")
    
    # 计算总文件大小
    total_elements = sum(emb.numel() for emb in train_embeddings_list)
    element_size = train_embeddings_list[0].element_size() if train_embeddings_list else 4
    file_size_mb = total_elements * element_size / 1024 / 1024
    print(f"✓ 文件大小: {file_size_mb:.1f} MB")

正在编码所有训练序列...


批量编码:   0%|          | 0/483 [00:00<?, ?it/s]

✓ 所有序列编码完成!
✓ 编码序列数量: 7720
✓ 长度统计: 最小=5, 最大=48, 平均=20.2
✓ 数据类型: torch.float32
✓ 数值范围: [-1.619, 1.542]

正在保存嵌入数据...
✓ 嵌入数据已保存到: embedding_amp.pt
✓ 文件大小: 610.4 MB


## 第5步：理解扩散过程基础

扩散模型的核心思想：
1. **前向过程**：逐步向干净数据添加噪声，直到变成纯噪声
2. **反向过程**：训练神经网络学习从噪声中恢复数据

我们需要理解：
- 噪声调度（sqrt schedule）
- 前向扩散公式
- 如何可视化噪声添加过程


## 第6步：构建UNet去噪网络

我们需要构建一个1D UNet来学习去噪过程：

**网络组件**：
- 时间嵌入（Time Embedding）：让模型知道当前处于哪个扩散步骤
- 残差块（ResBlock）：基本的卷积构建块
- 自注意力（Self-Attention）：在瓶颈层增强特征表达
- U型结构：编码器-瓶颈-解码器，带跳跃连接

**模型架构**：Trans-UNet风格的1D版本


## 第7步：训练扩散模型

训练过程的核心：
1. **随机采样时间步**：从0到2000中随机选择
2. **添加噪声**：根据时间步向原始数据添加相应强度的噪声
3. **模型预测**：UNet预测原始数据x0（而不是噪声）
4. **计算损失**：MSE损失，比较预测的x0和真实x0
5. **反向传播**：更新模型参数

**训练配置**：
- 扩散步数：2000步
- 预测目标：x0（原始数据）
- 噪声调度：sqrt schedule


## 第8步：DDPM采样生成新序列

从训练好的扩散模型采样生成新的嵌入向量：

**采样过程**：
1. **从纯噪声开始**：随机初始化(batch_size, 48, 1024)的噪声张量
2. **逐步去噪**：从时间步2000到0，逐步去除噪声
3. **下采样**：将2000步压缩到200步以加速采样
4. **噪声选择**：可以选择高斯噪声或均匀噪声

**DDPM算法**：使用标准的DDPM逆向采样公式


## 第9步：序列解码 - 将嵌入转换回氨基酸序列

使用ProtT5解码器将生成的嵌入向量转换回氨基酸序列：

**解码过程**：
1. **去除零填充**：识别并移除主要为零的行（padding部分）
2. **包装为编码器输出**：将嵌入包装成ProtT5期望的格式
3. **调用解码器**：使用T5的generate方法生成token序列
4. **后处理**：清理生成的文本，只保留20种标准氨基酸

**注意事项**：
- 解码可能不完美，需要过滤无效序列
- 生成的序列长度可能与原始序列不同


## 第10步：结果分析和可视化

分析生成的序列质量：

**质量评估**：
1. **基本统计**：序列长度分布、氨基酸组成
2. **与训练集对比**：比较生成序列和原始训练序列的特征
3. **有效性检查**：过滤掉无效或过短的序列
4. **多样性分析**：检查生成序列的多样性

**可视化**：
- 训练损失曲线
- 生成序列长度分布
- 氨基酸组成热图
- 训练集vs生成集对比


## 总结和下一步

### 🎯 我们完成的工作：

1. **数据准备**：准备了抗菌肽序列作为训练数据
2. **序列编码**：使用ProtT5编码器将氨基酸序列转换为1024维嵌入向量
3. **扩散建模**：理解了扩散过程的前向和反向过程
4. **网络架构**：构建了Trans-UNet风格的1D去噪网络
5. **模型训练**：训练扩散模型学习从噪声中恢复序列嵌入
6. **序列生成**：使用DDPM采样生成新的序列嵌入
7. **序列解码**：将嵌入转换回氨基酸序列
8. **结果评估**：分析生成序列的质量和多样性

### 🚀 改进方向：

1. **更多数据**：使用更大的AMP数据集训练
2. **模型优化**：调整网络架构和超参数
3. **条件生成**：根据特定性质（如长度、活性）生成序列
4. **质量筛选**：添加判别器或规则筛选高质量序列
5. **预训练微调**：实现论文中的预训练+微调策略

### 📚 进一步学习：

- 扩散模型的数学原理
- 蛋白质序列的生物学特性
- 抗菌肽的结构-功能关系
- 更高级的生成模型技术


In [ ]:
## 🛠️ 如果模型加载仍然失败，可以尝试以下手动解决方案：

# 方案1: 手动清理 transformers 缓存
def manual_cache_cleanup():
    """手动清理所有 transformers 缓存"""
    import shutil
    from transformers import TRANSFORMERS_CACHE
    
    print(f"Transformers 缓存目录: {TRANSFORMERS_CACHE}")
    
    if os.path.exists(TRANSFORMERS_CACHE):
        print("正在清理缓存...")
        try:
            shutil.rmtree(TRANSFORMERS_CACHE)
            print("✓ 缓存清理成功")
            os.makedirs(TRANSFORMERS_CACHE, exist_ok=True)
        except Exception as e:
            print(f"❌ 缓存清理失败: {e}")
    else:
        print("缓存目录不存在")

# 方案2: 检查网络连接
def test_huggingface_connection():
    """测试到 Hugging Face 的连接"""
    import requests
    
    urls_to_test = [
        "https://huggingface.co",
        "https://hf-mirror.com",
        "https://huggingface.co/Rostlab/prot_t5_xl_half_uniref50-enc"
    ]
    
    for url in urls_to_test:
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                print(f"✓ 连接成功: {url}")
            else:
                print(f"❌ 连接失败: {url} (状态码: {response.status_code})")
        except Exception as e:
            print(f"❌ 连接失败: {url} (错误: {e})")

# 方案3: 使用本地模型（如果已下载）
def try_local_model():
    """尝试使用本地已下载的模型"""
    from transformers import TRANSFORMERS_CACHE
    
    local_paths = [
        os.path.join(TRANSFORMERS_CACHE, "models--Rostlab--prot_t5_xl_half_uniref50-enc"),
        os.path.join(TRANSFORMERS_CACHE, "models--Rostlab--prot_t5_xl_uniref50"),
        "/root/.cache/huggingface/transformers/models--Rostlab--prot_t5_xl_half_uniref50-enc",
        "/root/.cache/huggingface/transformers/models--Rostlab--prot_t5_xl_uniref50"
    ]
    
    print("检查本地模型文件...")
    for path in local_paths:
        if os.path.exists(path):
            print(f"✓ 找到本地模型: {path}")
            # 列出文件
            try:
                files = os.listdir(path)
                print(f"  文件: {files[:10]}...")  # 显示前10个文件
            except:
                pass
        else:
            print(f"❌ 本地模型不存在: {path}")

print("=== 诊断工具 ===")
print("\n1. 测试网络连接:")
test_huggingface_connection()

print("\n2. 检查本地模型:")
try_local_model()

print("\n3. 如需清理缓存，取消注释下面的代码:")
print("# manual_cache_cleanup()")
